In [6]:
from datasets import load_dataset
food = load_dataset("food101", split="train[:5000]")

In [7]:
food = food.train_test_split(test_size=0.2)

In [8]:
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 'label': 6}

In [9]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
    

In [10]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [11]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [12]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
food = food.with_transform(transforms)

평가

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

훈련

In [16]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="food101_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

  5%|▌         | 10/186 [09:14<2:42:56, 55.55s/it]

{'loss': 4.5869, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.16}


 11%|█         | 20/186 [18:21<2:30:49, 54.52s/it]

{'loss': 4.2936, 'learning_rate': 4.970059880239521e-05, 'epoch': 0.32}


 16%|█▌        | 30/186 [27:36<2:21:28, 54.41s/it]

...

                                                   
100%|██████████| 186/186 [3:03:49<00:00, 55.13s/it]

{'eval_loss': 1.626529335975647, 'eval_accuracy': 0.872, 'eval_runtime': 295.5109, 'eval_samples_per_second': 3.384, 'eval_steps_per_second': 0.213, 'epoch': 2.98}


100%|██████████| 186/186 [3:03:50<00:00, 59.30s/it]

{'train_runtime': 11030.205, 'train_samples_per_second': 1.088, 'train_steps_per_second': 0.017, 'train_loss': 2.429398859700849, 'epoch': 2.98}


TrainOutput(global_step=186, training_loss=2.429398859700849, metrics={'train_runtime': 11030.205, 'train_samples_per_second': 1.088, 'train_steps_per_second': 0.017, 'train_loss': 2.429398859700849, 'epoch': 2.98})

추론

In [20]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]

In [21]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_awesome_food_model")
classifier(image)

OSError: my_awesome_food_model does not appear to have a file named config.json. Checkout 'https://huggingface.co/my_awesome_food_model/None' for available files.

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_awesome_food_model")
inputs = image_processor(image, return_tensors="pt")

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("my_awesome_food_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]